In [84]:
def solution(m):
    from fractions import Fraction
    from copy import deepcopy
    import numpy as np
    import math
    # Your code here
    
    m=np.array(m)
    m0=deepcopy(m)


    diag=deepcopy(m).diagonal()
    np.fill_diagonal(m, 0)

    # Index terminals:
    term=np.where(np.sum(m, axis=1)==0)[0]
    # P=0 terminals:
    p0term=np.where(np.sum(m, axis=0)==0)[0]



    nonterm=np.setdiff1d(list(range(len(m))),term)


    # Remove P=0 terminals from term list
    indp0=np.where(term==p0term)
    term=np.delete(term,indp0)    


    # prepare dict for terminal cases
    dm2=dict()
    for t in term:
        for rowi in range(len(m)):
            if m[rowi][t] >0:
                dm2.setdefault(t, []).append(rowi)





    dtrm=dict()
    for trm in term:
        trmd=np.nonzero(m[:,trm])[0]
        if len(trmd)>1:
            dtrm[trm]=deepcopy(m)[:,trm]
            m[:,trm]=0

    M=[]
    if len(dtrm)>0:
        for k in dtrm.keys():
            for i in range(len(dtrm[k])):
                if dtrm[k][i]>0:
                    mc=deepcopy(m)
                    val=deepcopy(dtrm[k][i])
                    mc[:,k][i]=val
                    M.append(mc)



    for k in dm2.keys():
        if len(dm2[k])>1:
            for v in dm2[k]:
                mx=deepcopy(m)
                mx[v][k]=0
                M.append(mx)
    if len(M)<2:
        M=[m]



    def hasInd(m_,nonterm, state):
        for i in nonterm:
            if state in np.nonzero(m_[i])[0]:
                return i
        return -1



    lstx2=[]
    for m_ in M:
        for t in term:
            lstx=[]
            while t>0:
                lstx.append(t)
                t=hasInd(m_,nonterm,t)
                if t==0:
                    lstx.append(t)
                    break
            lstx2.append(lstx)

    if len(lstx2)>1:
        roots=np.unique(lstx2)
        rootss=deepcopy(roots)
        roots=[]
        for root in rootss:
            if 0 in root:
                roots.append(root)

    else:
        roots=lstx2
    # get cyclic paths such as s0-->s1-->s0
    sets=[]
    cycles=[]
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            if set((i,j)) not in sets:
                sets.append(set((i,j))) 
                if m[i][j] != 0 and m[j][i] != 0:
                    cycles.append((i,j))

    for i in range(len(diag)):
        if diag[i]>0:
            cycles.append((i,i))

    np.fill_diagonal(m,diag)
    #print('roots: {}, cycles: {}'.format(roots, cycles))

    
    
    
    ####
    m=m0
    mprob=[m[i]/np.sum(m,axis=1)[i] for i in nonterm]

    # Get root probs

    roots[0]
    rootprobdict=dict()

    for root in roots:
        
        rootprobdict[root[0]]=[]

    probs=[]
    
    
    for root in roots:
        prob=[]
        for i in range(len(root)-1):
            prob.append(mprob[root[i+1]][root[i]])

        
      
        probs.append(prob)
        rootprobdict[root[0]].append(prob)
    #print(rootprobdict)
      
        
    def getCyclePath1(root, nonterm, cycles):
        cs=[]
        q=[]
        for r in root:
            if r in nonterm:# and r!=0:
                for c in cycles:
                    if r in c:# and 0 not in c:
                        cs.append(c)
                        if r==c[0]:# and c[1]!=0:
                            q.append(c[1])
                        else:#if c[0]!=0:
                            q.append(c[0])

        if 0 in q:
            q.remove(0)
        return q,cs
    
    def getCyclePath2(q,cs,cycles,root):
        qpl=[]
        while len(q)>0:
            qp=q.pop()
            for c in cycles:
                if qp in c:# and 0 not in c:
                    cs.append(c)
                    if qp==c[0]   and c[1] not in qpl: 
                        q.append(c[1])
                    elif c[0] not in qpl:
                        q.append(c[0])
                    qpl.append(qp)
        cs=set(cs)
        return cs
        

    
      # dict for roots
    rootdict=dict()
    for root in roots:
        rootdict[root[0]]=[]
    for root in roots:
        rootdict[root[0]].append(root)
        
        
      # dict for cycles per root
    cycrootdict=dict()
    for root in roots:
        cycrootdict[root[0]]=[]
    for root in roots:
        gcp1=getCyclePath1(root, nonterm, cycles)
        cpath=getCyclePath2(gcp1[0],gcp1[1],cycles,root)
        cycrootdict[root[0]].append(cpath)
        
        
        
      # dict for prob of cycles per root
    def getcycprob(cpath,mprob):
        cps=[]
        for cp in cpath:
            if cp[0]!=cp[1]:
                cps.append( mprob[cp[0]][cp[1]]*mprob[cp[1]][cp[0]])
            elif cp[0]==cp[1]:
                cps.append( mprob[cp[0]][cp[1]])
        return cps

    cycrootprobdict=dict()
    for root in roots:
        cycrootprobdict[root[0]]=[]
    for k in cycrootdict.keys():
        for v in cycrootdict[k]:
            cycrootprobdict[k].append(getcycprob(v,mprob))

                    
                    
    
    # calculate cyclic probs, as a series from 0 to 1e5 (pseudo infinite)                          
    def sumRepProb(A):
        return (A**(1e5+1)-1)/(A-1)
    
    def getFractions(rootprobdict,cycrootprobdict,roots):
        prod3=0
        # dictionary for final fractions
        fracdict=dict()
        for root in roots:
            fracdict[root[0]]=[]
        for k in rootprobdict:
            for lst in rootprobdict[k]:
                prod2=[]
                prod1=np.prod(lst)
                for v in cycrootprobdict[k]:
                    if len(v)>0:
                        prod2.append([sumRepProb(e) for e in v])
                    
                prod3=np.prod(prod2)
                fracdict[k].append(prod1*prod3)
        return fracdict
        
        
        
    fracdict=getFractions(rootprobdict,cycrootprobdict,roots)            
      
        
    
      
        
        
        
    # add up similar terminals - for when there are multiple paths to the same terminal

    for k in fracdict:
        fracdict[k]=np.sum(fracdict[k])
    for k in fracdict:
        z=fracdict[k]
        fracdict[k]=Fraction(*z.as_integer_ratio()).limit_denominator()







        # get lowest common denominator
    def returnInt(fraction):
        return (int(str(fraction).split('/')[0]),int(str(fraction).split('/')[1]))

    intfracdict=dict()
    for k in fracdict.keys():
        intfracdict[k]=returnInt(fracdict[k])




    denominators=dict()
    nominators=dict()
    for k in intfracdict.keys():
        denominators[k]=intfracdict[k][1]
        nominators[k]=intfracdict[k][0]


    # get 'orphan' states (p=0 terminals)

#     term2=list(set(list(term)+ list(p0term)    ))
#     print(term2)
    orphanterm=[]
    for c in range(m0.shape[1]):
        if sum(m0[:,c])==0 and sum(m0[c,:])==0:
            orphanterm.append(c)
    term2=list(set(list(term)+ orphanterm    ))

    
    for t in term2:
        if t not in intfracdict.keys():
            intfracdict[t]=0
    
      
    # lowest common denominator
    nominatorsL=list(nominators.values())
    denominatorsL=list(denominators.values())
    def getLcm(denominators): 
        lcm = denominators[0]
        for i in denominators[1:]:
            lcm = lcm*i//math.gcd(lcm, i)
        return(lcm)


    c=getLcm(denominatorsL)
    for i in range(len(denominatorsL)):
        nominatorsL[i]*=(int(c/denominatorsL[i]))
        
    nk=list(nominators.keys())
    nominatorsdict2=dict()
    for i in range(len(nk)):
        nominatorsdict2[nk[i]]=nominatorsL[i]

        
    LCM=getLcm(list(denominators.values()))


    result=[]
    for t in term2:
        if t in nominatorsdict2.keys():
            result.append(nominatorsdict2[t])
        else:
            result.append(0)
        
    result.append(LCM)
    return result


In [86]:
m=[
[0,1,0,1,0,0],
[4,0,0,3,2,0],
[3,1,3,1,1,0],
[4,1,1,5,1,0],
[0,0,0,0,0,0],
[0,0,0,0,0,0]]
m=([[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0,0], [0, 0, 0, 0, 0]])
m=[[0,1,0,0,0,1],
   [4,0,0,3,2,0],
   [0,0,0,0,0,0],
   [0,0,0,0,0,0],
   [0,0,0,0,0,0],
   [0,0,0,0,0,0]]
solution(m)

[0, 3, 2, 9, 14]

In [89]:
solution([[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0,0], [0, 0, 0, 0, 0]])

[7, 6, 8, 21]

In [97]:
m=   [
            [0, 1, 0, 0, 0, 1],
            [1, 0, 0, 1, 1, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0]
        ]
solution(m)

[0, 1, 1, 3, 5]